### Deliverable 1: Preprocessing the Data for a Neural Network

In [8]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
vax_df = pd.read_csv("sample_data/vax.csv")
demo_df = pd.read_csv("sample_data/demo.csv")
vax_df.head()
demo_df.head()

,Location,County,Fips,Total_Pop,Race/Ethnicity - Asian/NHPI,Race/Ethnicity - Black,Race/Ethnicity - Hispanic/Latino,Race/Ethnicity - White,"Income Level - $100,000 to $199,999","Income Level - $15,000 to $24,999","Income Level - $200,000 or more","Income Level - $25,000 to $34,999","Income Level - $35,000 to $49,999","Income Level - $50,000 to $74,999","Income Level - $75,000 to $99,999","Income Level - Under $15,000"
0,Bronx,Bronx,36005,1402855.40,57931.72,411825.93,805111.42,127986.33,71046,60886,16758,46775,63007,86831,49307,119280
1,Brooklyn,Kings,36047,2505488.00,319915.00,760626.00,483679.00,941268.00,224810,84707,111304,74789,97196,142753,110588,131944
2,Manhattan,New York,36061,1575266.24,202866.40,199900.07,407864.84,764634.93,177697,47553,192633,40180,49280,81461,74802,104597
3,Queens,Queens,36081,2213259.38,601751.88,408815.01,638296.75,564395.74,213534,51679,72801,62055,82356,127845,101044,73238
4,Staten Island,Richmond,36085,468187.00,51177.00,44681.00,88638.00,283691.00,53693,9822,22941,9351,17346,24378,16794,11972


In [9]:
# join files
df = vax_df.merge(demo_df, how='left',on='County')
df.head()

,County,First Dose,Series Complete,Report as of,Location,Fips,Total_Pop,Race/Ethnicity - Asian/NHPI,Race/Ethnicity - Black,Race/Ethnicity - Hispanic/Latino,Race/Ethnicity - White,"Income Level - $100,000 to $199,999","Income Level - $15,000 to $24,999","Income Level - $200,000 or more","Income Level - $25,000 to $34,999","Income Level - $35,000 to $49,999","Income Level - $50,000 to $74,999","Income Level - $75,000 to $99,999","Income Level - Under $15,000"
0,Bronx,8195,29,1/1/21,Bronx,36005,1402855.40,57931.72,411825.93,805111.42,127986.33,71046,60886,16758,46775,63007,86831,49307,119280
1,Kings,15690,64,1/1/21,Brooklyn,36047,2505488.00,319915.00,760626.00,483679.00,941268.00,224810,84707,111304,74789,97196,142753,110588,131944
2,New York,20138,71,1/1/21,Manhattan,36061,1575266.24,202866.40,199900.07,407864.84,764634.93,177697,47553,192633,40180,49280,81461,74802,104597
3,Queens,16859,63,1/1/21,Queens,36081,2213259.38,601751.88,408815.01,638296.75,564395.74,213534,51679,72801,62055,82356,127845,101044,73238
4,Richmond,4959,14,1/1/21,Staten Island,36085,468187.00,51177.00,44681.00,88638.00,283691.00,53693,9822,22941,9351,17346,24378,16794,11972


In [24]:
# Keep most recent dates
clean_df = pd.DataFrame(df.loc[df['Report as of'] == '10/22/21'])
clean_df

,County,First Dose,Series Complete,Report as of,Location,Fips,Total_Pop,Race/Ethnicity - Asian/NHPI,Race/Ethnicity - Black,Race/Ethnicity - Hispanic/Latino,Race/Ethnicity - White,"Income Level - $100,000 to $199,999","Income Level - $15,000 to $24,999","Income Level - $200,000 or more","Income Level - $25,000 to $34,999","Income Level - $35,000 to $49,999","Income Level - $50,000 to $74,999","Income Level - $75,000 to $99,999","Income Level - Under $15,000"
1470,Bronx,973636,857631,10/22/21,Bronx,36005,1402855.40,57931.72,411825.93,805111.42,127986.33,71046,60886,16758,46775,63007,86831,49307,119280
1471,Kings,1697592,1523260,10/22/21,Brooklyn,36047,2505488.00,319915.00,760626.00,483679.00,941268.00,224810,84707,111304,74789,97196,142753,110588,131944
1472,New York,1360459,1200406,10/22/21,Manhattan,36061,1575266.24,202866.40,199900.07,407864.84,764634.93,177697,47553,192633,40180,49280,81461,74802,104597
1473,Queens,1844716,1673837,10/22/21,Queens,36081,2213259.38,601751.88,408815.01,638296.75,564395.74,213534,51679,72801,62055,82356,127845,101044,73238
1474,Richmond,327279,296750,10/22/21,Staten Island,36085,468187.00,51177.00,44681.00,88638.00,283691.00,53693,9822,22941,9351,17346,24378,16794,11972


In [25]:
# Calculate Vax Rate
clean_df['Vax_Rate'] = clean_df['Series Complete'] / clean_df['Total_Pop']
clean_df

,County,First Dose,Series Complete,Report as of,Location,Fips,Total_Pop,Race/Ethnicity - Asian/NHPI,Race/Ethnicity - Black,Race/Ethnicity - Hispanic/Latino,Race/Ethnicity - White,"Income Level - $100,000 to $199,999","Income Level - $15,000 to $24,999","Income Level - $200,000 or more","Income Level - $25,000 to $34,999","Income Level - $35,000 to $49,999","Income Level - $50,000 to $74,999","Income Level - $75,000 to $99,999","Income Level - Under $15,000",Vax_Rate
1470,Bronx,973636,857631,10/22/21,Bronx,36005,1402855.40,57931.72,411825.93,805111.42,127986.33,71046,60886,16758,46775,63007,86831,49307,119280,0.611347
1471,Kings,1697592,1523260,10/22/21,Brooklyn,36047,2505488.00,319915.00,760626.00,483679.00,941268.00,224810,84707,111304,74789,97196,142753,110588,131944,0.607969
1472,New York,1360459,1200406,10/22/21,Manhattan,36061,1575266.24,202866.40,199900.07,407864.84,764634.93,177697,47553,192633,40180,49280,81461,74802,104597,0.762034
1473,Queens,1844716,1673837,10/22/21,Queens,36081,2213259.38,601751.88,408815.01,638296.75,564395.74,213534,51679,72801,62055,82356,127845,101044,73238,0.756277
1474,Richmond,327279,296750,10/22/21,Staten Island,36085,468187.00,51177.00,44681.00,88638.00,283691.00,53693,9822,22941,9351,17346,24378,16794,11972,0.633828


In [26]:
# Drop the non-beneficial columns, County, First Dose, Series Complete, Report as of, Location
clean_df = clean_df.drop(['County', 'First Dose', 'Series Complete', 'Report as of', 'Location'], axis=1)
clean_df.head()

,Fips,Total_Pop,Race/Ethnicity - Asian/NHPI,Race/Ethnicity - Black,Race/Ethnicity - Hispanic/Latino,Race/Ethnicity - White,"Income Level - $100,000 to $199,999","Income Level - $15,000 to $24,999","Income Level - $200,000 or more","Income Level - $25,000 to $34,999","Income Level - $35,000 to $49,999","Income Level - $50,000 to $74,999","Income Level - $75,000 to $99,999","Income Level - Under $15,000",Vax_Rate
1470,36005,1402855.40,57931.72,411825.93,805111.42,127986.33,71046,60886,16758,46775,63007,86831,49307,119280,0.611347
1471,36047,2505488.00,319915.00,760626.00,483679.00,941268.00,224810,84707,111304,74789,97196,142753,110588,131944,0.607969
1472,36061,1575266.24,202866.40,199900.07,407864.84,764634.93,177697,47553,192633,40180,49280,81461,74802,104597,0.762034
1473,36081,2213259.38,601751.88,408815.01,638296.75,564395.74,213534,51679,72801,62055,82356,127845,101044,73238,0.756277
1474,36085,468187.00,51177.00,44681.00,88638.00,283691.00,53693,9822,22941,9351,17346,24378,16794,11972,0.633828


In [27]:
# Check all values are numeric
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 1470 to 1474
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Fips                                 5 non-null      int64  
 1   Total_Pop                            5 non-null      float64
 2   Race/Ethnicity - Asian/NHPI          5 non-null      float64
 3   Race/Ethnicity - Black               5 non-null      float64
 4   Race/Ethnicity - Hispanic/Latino     5 non-null      float64
 5   Race/Ethnicity - White               5 non-null      float64
 6   Income Level - $100,000 to $199,999  5 non-null      int64  
 7   Income Level - $15,000 to $24,999    5 non-null      int64  
 8   Income Level - $200,000 or more      5 non-null      int64  
 9   Income Level - $25,000 to $34,999    5 non-null      int64  
 10  Income Level - $35,000 to $49,999    5 non-null      int64  
 11  Income Level - $50,000 to $74,

In [28]:
# Split our preprocessed data into our features and target arrays
y = clean_df["Vax_Rate"].values
X = clean_df.drop(["Vax_Rate"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [30]:
# Update for linear regression

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, 
                             activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                300       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 741
Trainable params: 741
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="mean_absolute_error", 
           optimizer="adam", 
           metrics=["accuracy"])

In [33]:
# Create a callback that saves the model's weights every 5 epochs
callbacks = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_weights_only=True, save_freq=5)

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[callbacks])

Epoch 1/100
1/1 [==============================] - 1s 829ms/step - loss: 0.8660 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 7ms/step - loss: 0.8136 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 0.7612 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 7ms/step - loss: 0.7091 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 19ms/step - loss: 0.6569 - accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6048 - accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 0.5531 - accuracy: 0.0000e+00
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 0.5024 - accuracy: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 0.4515 - accuracy: 0.0000e+00
Epoch 10/100
1/1 [==============================] - 0s 22ms/step - loss: 0.4004 - accura

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.5036 - accuracy: 0.0000e+00
Loss: 0.5035684108734131, Accuracy: 0.0


In [36]:
#Save 
nn.save('COVID_Vax_Predictions.h5')